In [ ]:
def load_sps(resolution, resolution_wave, zred):
    """Read the SSPs into a CSPSpecBasis object.
    
    Also smooth all the models to the instrumental resolution of the 
    
    """
    
    t0 = time.time()
    print('Initializing CSPSpecBasis...')
    sps = CSPSpecBasis(zcontinuous=1, compute_vega_mags=False, vactoair_flag=False)
    print('...took {:.1f} seconds.'.format(time.time() - t0)) 
    
    sps.csp.params['smooth_lsf'] = True
    sps.csp.set_lsf(wave, delta_v)


    restwave = resolution_wave / (1 + zred)
    delta_v = 2.998e5 * resolution / restwave
    
def get_lsf(spec, miles_fwhm_aa=2.54, zred=0.0, **extras):
    """This method takes a spec file and returns the quadrature difference
    between the instrumental dispersion and the MILES dispersion, in km/s, as a
    function of wavelength
    """
    lightspeed = 2.998e5  # km/s
    # Get the SDSS instrumental resolution for this plate/mjd/fiber
    wave_obs = 10**spec['loglam']  # observed frame wavelength
    # This is the instrumental velocity resolution in the observed frame
    sigma_v = np.log(10) * lightspeed * 1e-4 * spec['wdisp']
    # filter out some places where sdss reports zero dispersion
    good = sigma_v > 0
    wave_obs, sigma_v = wave_obs[good], sigma_v[good]
    # Get the miles velocity resolution function at the corresponding
    # *rest-frame* wavelength
    wave_rest = wave_obs / (1 + zred)
    sigma_v_miles = lightspeed * miles_fwhm_aa / 2.355 / wave_rest

    # Get the quadrature difference
    # (Zero and negative values are skipped by FSPS)
    dsv = np.sqrt(np.clip(sigma_v**2 - sigma_v_miles**2, 0, np.inf))
    # Restrict to regions where MILES is used
    good = (wave_rest > 3525.0) & (wave_rest < 7500)

    # return the broadening of the rest-frame library spectra required to match
    # the obserrved frame instrumental lsf
    return wave_rest[good], dsv[good]
    
    
    
    sps.ssp.params['smooth_lsf'] = True
    sps.ssp.set_lsf(restwave, delta_v)
    
    return sps